# Использование питоновского модуля Pathlib

> Оригинал статьи Криса [тут](https://pbpython.com/pathlib-intro.html)

## Введение

Сложно написать скрипт на Python, который не взаимодействует с файловой системой. Действие может быть таким же простым, как чтение файла данных в DataFrame, или таким сложным, как анализ тысяч файлов в глубоко вложенной структуре каталогов. Стандартная библиотека Python имеет несколько полезных функций для этих задач, включая модуль pathlib.

Модуль pathlib был впервые включен в python 3.4 и улучшался в каждом из последующих выпусков. Pathlib - это объектно-ориентированный интерфейс к файловой системе, обеспечивающий более интуитивно понятный метод взаимодействия с файловой системой независимо от платформы и питоническим способом.

Недавно у меня был небольшой проект, в котором я решил использовать pathlib в сочетании с pandas для сортировки и управления тысячами файлов во вложенной структуре каталогов. Как только все это было реализовано, я действительно оценил возможности, которые предоставляет pathlib, и обязательно буду использовать его в будущих проектах. Этот проект послужил источником вдохновения для этой статьи.

Чтобы помочь другим, я создал шпаргалку по pathlib, которая, надеюсь, упростит использование этой замечательной библиотеки. Позже в этом посте я приведу пример построения кадра данных pandas на основе структуры каталогов. Это полезный инструмент для просмотра и анализа большого количества файлов - особенно в системе Windows, где весь спектр инструментов оболочки недоступен.

## Начало работы с Pathlib

Библиотека pathlib включена во все версии python> = 3.4. Я рекомендую использовать последнюю версию python, чтобы получить доступ ко всем последним обновлениям. В этой статье я буду использовать python 3.8.

Одна из полезных функций модуля pathlib заключается в том, что создание путей без использования os.joindir более интуитивно понятно. Например, когда я запускаю небольшие проекты, я создаю in и out каталоги как подкаталоги в текущем рабочем каталоге (с помощью os.getcwd()). Я использую эти каталоги для хранения рабочих файлов ввода и вывода. Вот как будет выглядеть этот код:

In [1]:
import os

in_dir = os.path.join(os.getcwd(), "in")
out_dir = os.path.join(os.getcwd(), "out")
in_file = os.path.join(in_dir, "input.xlsx")
out_file = os.path.join(out_dir, "output.xlsx")

Это работает, но немного неуклюже. Например, если бы я хотел определить только входные и выходные файлы без определения каталогов, это выглядело бы так:

In [2]:
import os

in_file = os.path.join(os.path.join(os.getcwd(), "in"), "input.xlsx")
out_file = os.path.join(os.path.join(os.getcwd(), "out"), "output.xlsx")

Хммм. Это несложно, но определенно некрасиво.

Давайте посмотрим, как это будет выглядеть, если мы воспользуемся модулем pathlib.

In [3]:
from pathlib import Path

in_file_1 = Path.cwd() / "in" / "input.xlsx"
out_file_1 = Path.cwd() / "out" / "output.xlsx"

Интересно. На мой взгляд, это гораздо легче мысленно разобрать. Это аналогично методу os.path присоединения текущего рабочего каталога (с помощью Path.cwd()) к различным подкаталогам и расположениям файлов. Гораздо легче следовать из-за умного переопределения / для создания пути более естественным образом, чем связывание множества os.path.joins вместе.

Кроме того, если вам не нравится приведенный выше синтаксис, вы можете связать несколько частей вместе, используя joinpath:

In [4]:
in_file_2 = Path.cwd().joinpath("in").joinpath("input.xlsx")
out_file_2 = Path.cwd().joinpath("out").joinpath("output.xlsx")

На мой взгляд, это немного сложнее, но все же намного лучше, чем os.path.join безумие выше.

Наконец, есть еще один прием, который можно использовать для создания пути с несколькими каталогами:

In [5]:
parts = ["in", "input.xlsx"]
in_file_3 = Path.cwd().joinpath(*parts)

Независимо от того, какой метод вы используете, эти подходы работают для построения пути к файлу или каталогу. Дополнительным преимуществом этих методов является то, что вы создаете объект Path, а не просто строковое представление пути. Посмотрите на разницу между печатью in_file и in_file_1

In [6]:
print(in_file)
print(type(in_file))

/home/dmf/new_github/анализ_данных/pandas_basic/быстрое введение в pandas/in/input.xlsx
<class 'str'>


Результатом os.path.join является обычная строка. Сравните это с различными подходами к pathlib:

In [7]:
print(in_file_1)
print(type(in_file_1))

/home/dmf/new_github/анализ_данных/pandas_basic/быстрое введение в pandas/in/input.xlsx
<class 'pathlib.PosixPath'>


Фактическое строковое представление то же самое, но тип переменной - pathlib.PosixPath. Тот факт, что путь является объектом, означает, что мы можем выполнять множество полезных действий с объектом. Также интересно, что объект пути "знает", что он находится в системе Linux (также известной как Posix), и внутренне представляет его таким образом без необходимости сообщать программисту. Преимущество заключается в том, что код будет работать одинаково на машине с Windows и что базовая библиотека позаботится о любых чудесах Windows.

## Работа с объектами Path

Теперь, когда вы знаете основы создания объекта Path, давайте посмотрим, что мы можем сделать с этим объектом. В этой статье я буду использовать простую вложенную структуру, которая содержит файлы CSV и Excel и хранится на внешнем USB-накопителе. Вот как это выглядит в системе Linux:

![](https://pbpython.com/images/pathlib-tree.png)